# Numeriske metoder

Fokuset i dette emnet er numeriske løsninger. Vi begynner derfor med en innføring i "hva og hvordan" angående numeriske løsninger, og diskuterer numeriske evalueringer av de partiellderiverte, som er grunnsteinen som alle våre metoder baserer seg på.

## 1. Innføring

Uten en slags formel eller en algoritme, så trenger vi uendelig mye data til å beskrive en funksjon $\mathbb{R}\rightarrow\mathbb{R}$.

Likevel kommer vi langt hvis funksjonen er deriverbar, og vi kjenner en endelig mengde med verdier $(x_i, f(x_i))$.

### Eksempel 1: vi tegner en graf

Ta en titt på koden under og hva som skjer når vi tegner grafen til $f(x)=\sin(\pi x)$. For å lage grafen har vi altså ikke brukt noe kunnskap om funksjonen $\sin(x)$ utenom de $100$ verdiene $\sin(x_i)$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# lag 100 punkter mellom 0 og 1
x = np.linspace(0,1,100)

# beregn f(x) på alle punktene
f = np.sin(np.pi * x)

# lager en plott fra de 100 parene (x,f(x))
plt.plot(x,f)

### Eksempel 2: vi plotter en overflate i 3d
   
På samme måte, hvis vi har en funksjon $\mathbb{R}^2\rightarrow\mathbb{R}$ av to variabler, er det ofte nok å ha en mengde med verdier $\big(x_j, y_i, f(x_j, y_i)\big)$.   


*Hvorfor kommer $j$ før $i$? Se diskusjon som kommer etter kodefeltet/plottet*

Når vi plotter funksjoner for eksempel, lager vi et rutenett av punkter $(x_j, y_i)$ og beregner funksjonsverdiene $f(x_j, y_i)$. Ingen annen informasjon brukes:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# vi lager verdiene for x og y med 100 verdier for hver
x = np.linspace(-1,1,100)
y = np.linspace(-1,1,100)

# lager rutenettet med 100x100 = 10.000 punkter
X, Y = np.meshgrid(x,y)

# finn sin(pi*x*y) på disse 10.000 punktene
Z = np.sin(np.pi * X * Y)

# plotter de 10.000 funksjonsverdiene
fig,ax2 = plt.subplots(subplot_kw ={"projection":"3d"}, figsize=(10,8))
ax2.plot_surface(X, Y, Z)

plt.show()

### Advarsel: arrayer, rutenett, og meshgrid

Du lurte kanskje på hvorfor vi brukte $f(x_j,y_i)$, og ikke $f(x_i,y_j)$?

Når vi skriver matriser kommer tradisjonelt radene først, dvs. tallet $A[i,j]$ er i rad $i$ og kolonne $j$. 

Men når vi skal tegne grafen til en funksjon $f(x,y)$, er det vanligere å la $x$-aksen gå horisontalt og $y$-aksen vertikalt. Det mest naturlige rutenettet har derfor $x$-retning tilsvarende kolonner, og $y$ tilsvarende rader. Vi setter derfor opp

$$
F[i,j] = f(x_j, y_i)
$$

Numpy sin meshgrid-funksjon lager som default et rutenett $X[i,j], Y[i,j]$ slik at $X[i,j]=x[j]$ og $Y[i,j]=y[i]$. Resultatet er at $F = F(X,Y)$ gir arrayen over.

Det er imidlertidig mulig å overstyre slik at man heller får $F[i,j]=f(x_i,y_j)$ hvis man foretrekker det. Du kan prøve dette i kodefeltet under.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# lager 5 punkter mellom -1 og 0 for x og mellom 0 og 1 for y
x = np.linspace(-1,0,5)
y = np.linspace(0,1,5)

# lager et rutenett med 5x5 = 25 punkter
X1, Y1 = np.meshgrid(x,y)

# lager samme rutenett, med motsett rekkefølge
X2, Y2 = np.meshgrid(x,y,indexing='ij')

# print ut X-verdiene i den vanlige - de følger den vanlige x-aksen
print(X1)

# hvis du printer ut X-verdiene for den andre muligheten, vil du kansje ikke få hva du forventet 
# print(X2)

## 2. Diskretisering og de partiellderiverte

Hva med de partiellderiverte? Hvordan skal vi finne dem dersom vi bare vet noen funksjonsverdier $f(x_j, y_i)$?

### a) Repetisjon: Numerisk derivasjon

Husk fra ProgNumSikk(PNS) at hvis vi har en funksjon $f(x)$ av én variabel, så finnes det tre naturlige måter å tilnærme den deriverte $f'(x)$ på:

1. Forlengs differanser

$$
f'(x) \approx \frac{f(x+h)-f(x)}{h}
$$

2. Baklengs differanser

$$
f'(x) \approx \frac{f(x)-f(x-h)}{h}
$$

3. Sentrale differanser

$$
f'(x) \approx \frac{f(x+h)-f(x-h)}{2h}
$$

Det er ingen hindring i å bruke disse formlene om vi kun vet funksjonsverdiene på enkelte punkter $f(x_i)$. Det vi gjør er å la $h$ være avstanden mellom tilstøtende punkter, dvs:

1. Forlengs differanser

$$
f'(x_i) \approx \frac{f(x_{i+1})-f(x_i)}{h}
$$


2. Baklengs differanser

$$
f'(x_i) \approx \frac{f(x_i)-f(x_{i-1})}{h}
$$

3. Sentrale differanser
$$
f'(x_i) \approx \frac{f(x_{i+1})-f(x_{i-1})}{2h}
$$

I formlene har vi antatt at avstanden $h$ mellom punktet $x_i$ og nabopunktene, $x_{i-1}$ og $x_{i+1}$ er konstant, altså at $x_{i+1} = x_i + h$ for alle $i$. 

Vi viser hvordan dette fungerer under.

### Oppgave

Hvilken metode har vi tatt i bruk i koden under? Kan du endre koden til å kjøre en av de andre?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# lag 100 punkter mellom 0 og 1 langs x-aksen
x = np.linspace(0,1,100)

# vi regner f(x) på alle punktene
f = np.sin(np.pi * x)

# vi lager df hvor df[i] = f[i+1]-f[i]
# funksjonen np.diff() kan også brukes
df = f[1:] - f[:-1] 

# vi lager dx hvor dx[i] = x[i+1] - x[i]
# (akkurat her så kommer alle elementene til å være h)
dx = x[1:] - x[:-1]

# vi lager den deriverte, dvs. g[i] = df[i]/dx[i]
g = df / dx

# vi har kun 99 punkter siden np.diff(f) må stoppe utregningen for df[i] = f[i+1] - f[i] ved i=98.
plt.plot(x[:-1],g)

### b) Numerisk partiellderivasjon

Partiellederivasjon fungerer som over, bare at vi nå har et rutenett med punkter $(x_j,y_i)$.

La $h$ være avstanden mellom punktene i $x$-retning. Vi har da følgende mulige beregninger av $f_x$:

1. Forlengs differanser

$$
f_x(x_j,y_i) \approx \frac{f(x_{j+1},y_i)-f(x_j,y_i)}{h}
$$


2. Baklengs differanser

$$
f_x(x_j, y_i) \approx \frac{f(x_j,y_i)-f(x_{j-1},y_i)}{h}
$$

3. Sentrale differanser
$$
f_x(x_j, y_i) \approx \frac{f(x_{j+1},y_i)-f(x_{j-1},y_i)}{2h}
$$

La nå $k$ være avstanden mellom punktene i $y$-retning. For $f_y$ får vi

1. Forlengs differanser

$$
f_y(x_j,y_i) \approx \frac{f(x_j,y_{i+1})-f(x_j,y_i)}{k}
$$


2. Baklengs differanser

$$
f_y(x_j,y_i) \approx \frac{f(x_j,y_{i})-f(x_j,y_{i-1})}{k}
$$

3. Sentrale differanser

$$
f_y(x_j,y_i) \approx \frac{f(x_j,y_{i+1})-f(x_j,y_{i-1})}{2k}
$$

Vi viser hvordan dette gjøres under.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# vi lager verdiene for x og y
x = np.linspace(-1,1,100)
y = np.linspace(-1,1,100)

# vi lager rutenettet med 100x100 = 10.000 punkter
X, Y = np.meshgrid(x,y)

# regner sin(pi*x*y) på disse 10.000 punktene
Z = np.sin(np.pi * X * Y)

# vi tar differansene langs x-aksen, dZx[i,j] = Z[i,j+1] - Z[i,j]
# j-koordinat tilsvarer x, se kommentaren  "Advarsel: Arrayer, rutenett og meshgrid"
dZx = Z[:,1:] - Z[:,:-1]  # dZx = np.diff(Z, axis=1)  kunne også blitt brukt

# tar differansen dX[i,j] = X[i,j+1]-X[i,j] langs y-aksen
dX = X[:,1:] - X[:,:-1]

# vi regner den deriverte i x-retning 
# vi kan erstatte dX med et tall h hvis avstanden mellom punktene i x er konstant lik h, slik de er her 
Zx = dZx / dX

# Tilsvarende beregning i y-retning
dZy = Z[1:,:] - Z[:-1,:]
dY = Y[1:,:] - Y[:-1,:]  # Y = np.diff(Y, axis=0) kunne også blitt brukt
Zy = dZy / dY

# Vi plotter f_x til venstre og f_y til høyre
# vi må bruke X[:,0:-1] og tilsvarende på Y siden vi mistet en x-verdi da vi brukte np.diff
fig,(ax1, ax2) = plt.subplots(1,2,subplot_kw ={"projection":"3d"}, figsize=(15,10))
ax1.plot_surface(X[:,:-1], Y[:,:-1], Zx)
ax2.plot_surface(X[:-1,:], Y[:-1,:], Zy)
plt.show()